# Run multiple robots for one day

Run an experiment with multiple robots exploring an environment during the course of one day. 

The specification of the experiment is fully determined by the exp/run of type nrobot1day. This exp/run will refer to other exp/runs to specify the environment, the estimator and the scoring function. It will also list the names of the robots, their policies and other information. 

In [1]:
import sys
sys.path.append("..")
from settings import Config
import pathlib
from pprint import pprint


from wbf_simulate import simulate_1day_multirobot, save_simulation_results
from wbf_helper import get_geometry, create_wbfe, create_policy, create_estimator, create_score
from communication import PerfectCommunicationMedium, Message
from policy import RandomWaypointPolicy, AbstractCommunicateAndFollowPath

from robot import Robot

In [2]:
# the experiment for the robot run
experiment = "nrobot1day"
# run = "two-random-robots"
# run = "two-communicating-robots"
# run = "mrmr-3random"
# run = "mrmr-3lawnmowers"
run = "mrmr-lead-2bidders"

exp = Config().get_experiment(experiment, run)
pprint(exp)

resultsfile = pathlib.Path(exp["data_dir"], "results.pickle")
if resultsfile.exists():
    print(f"Results file already exists:\n{resultsfile}")
    print(f"Delete this file if re-running is desired.")
    raise Exception("Nothing to do.")

# the exp for the environment
exp_env = Config().get_experiment("environment", exp["exp_environment"])
pprint(exp_env)
# the exp for estimator
exp_estimator = Config().get_experiment("estimator", exp["exp_estimator"])
pprint(exp_estimator)
# the exp for the score
exp_score = Config().get_experiment("score", exp["exp_score"])
pprint(exp_score)

# FIXME: implement me
# the exp/runs for the policy for policies
# exp_policy = Config().get_experiment("policy", exp["exp_policy"])
# pprint(exp_policy)
robotspecs = []
for values in exp["robots"]:
    robotspec = {}
    robotspec["name"] = values["name"]
    robotspec["exp-policy"] = Config().get_experiment("policy", values["exp-policy"]) 
    robotspecs.append(robotspec)
pprint(robotspecs)


Loading pointer config file: C:\Users\lboloni\.config\WaterBerryFarms\mainsettings.yaml
Loading machine-specific config file: G:\My Drive\LotziStudy\Code\PackageTracking\WaterBerryFarms\settings\WBF-config-LotziYoga.yaml
No system dependent experiment file
 G:\My Drive\LotziStudy\Code\PackageTracking\WaterBerryFarms\settings\experiment-config\LotziYoga\nrobot1day\mrmr-lead-2bidders_sysdep.yaml,
 that is ok, proceeding.
Configuration for experiment: nrobot1day/mrmr-lead-2bidders successfully loaded
{'data_dir': WindowsPath('C:/Users/lboloni/Documents/Code/_TempData/WBF/ExperimentData/nrobot1day/mrmr-lead-2bidders'),
 'exp_environment': 'miniberry-100',
 'exp_estimator': 'adaptive-disk',
 'exp_run_sys_indep_file': WindowsPath('C:/Users/lboloni/Documents/Code/_Checkouts/WBF/WaterberryFarms/experiment_configs/nrobot1day/mrmr-lead-2bidders.yaml'),
 'exp_score': 'weighted_asymmetric',
 'group_name': 'nrobot1day',
 'im_resolution': 10,
 'name': '1 lead 2 bidders',
 'robots': [{'exp-policy': '

In [3]:
# FIXME: this is an experiment for the communicating agents, it 
# does not belong here

class SimpleCommunicator(AbstractCommunicateAndFollowPath):
    def __init__(self, exp_policy, exp_env):
        waypoints = [[5, 5 * exp_policy["seed"]]]
        repeat = False
        vel = 1
        super().__init__(vel, waypoints, repeat)
        name = exp_policy["policy-name"]
    
    def act_send(self, round):
        print(f"{self.name} act_send called at round {round}")
        self.robot.com.send(self.robot, destination=None, message = Message("hello"))
        
    def act_receive(self, round, messages):
        print(f"{self.name} act_receive called at round {round}")
        print(f"Messages {messages}")


In [4]:
# FIXME: this is an experiment for the communicating agents, it 
# does not belong here

def generateLocalCommunicator(exp_policy, exp_env):
    """Example of how to generate a policy with a local specification. For the time being, we are creating a random waypoint one..."""
    #
    #  Random waypoint policy
    #
    #print("Generate Local Communicator with spec {exp_policy}")
    #geo = get_geometry(exp_env["typename"])
    #policy = RandomWaypointPolicy(
    #     vel = 1, low_point = [geo["xmin"], 
    #     geo["ymin"]], high_point = [geo["xmax"], geo["ymax"]], seed = exp_policy["seed"])  
    # policy.name = exp_policy["policy-name"]
    policy = SimpleCommunicator(exp_policy, exp_env)
    return policy



In [5]:
results = {}
results["estimator-CODE"] = create_estimator(exp_estimator, exp_env)
results["estimator-name"] = results["estimator-CODE"].name
results["score-code"] = create_score(exp_score, exp_env)
results["score-name"] = results["score-code"].name

results["velocity"] = exp["velocity"]
results["timesteps-per-day"] = exp["timesteps-per-day"]
# results["timesteps-per-day-override"] = exp["timesteps-per-day-override"]
results["time-start-environment"] = exp["time-start-environment"]
results["im_resolution"] = exp["im_resolution"]
results["results-basedir"] = exp["data_dir"]
results["action"] = "run-one-day"
results["typename"] = exp_env["typename"]
wbf, wbfe = create_wbfe(exp_env)
# move ahead to the starting point of the environment
wbfe.proceed(results["time-start-environment"])
results["wbf"] = wbf
results["wbfe"] = wbfe
results["days"] = 1
com = PerfectCommunicationMedium(wbfe)
results["communication"] = com
results["communication-rounds"] = 5
get_geometry(results["typename"], results)

robots = []
for robotspec in robotspecs:
    # create the robot and set the policy
    robot = Robot(robotspec["name"], 0, 0, 0, env=None, im=None)
    robot.com = results["communication"]
    if robotspec["exp-policy"]["policy-code"] == "-":
        generator = robotspec["exp-policy"]["policy-code-generator"]        
        policy = eval(generator)(robotspec["exp-policy"], exp_env)
    else:
        policy = create_policy(robotspec["exp-policy"], exp_env)
    robot.assign_policy(policy)
    robots.append(robot)
    com.add_robot(robot)
results["robots"] = robots

# 
# This is where we actually calling the simulation
#
simulate_1day_multirobot(results)
#print(f"Saving results to: {resultsfile}")
#with compress.open(resultsfile, "wb") as f:
#    pickle.dump(results, f)
save_simulation_results(resultsfile, results)

INFO:root:Environment.proceed - calling the inner_proceed
INFO:root:Environment.proceed - calling the inner_proceed
INFO:root:PrecalculatedEnvironment at timestamp 25
INFO:root:Loading from gz C:\Users\lboloni\Documents\Code\_TempData\WBF\ExperimentData\environment\miniberry-100\precalc_tylcv\env_value_00025.gz
INFO:root:Loading from gz C:\Users\lboloni\Documents\Code\_TempData\WBF\ExperimentData\environment\miniberry-100\precalc_tylcv\env_value_00025.gz done
INFO:root:Environment.proceed - calling the inner_proceed
INFO:root:PrecalculatedEnvironment at timestamp 25
INFO:root:Loading from gz C:\Users\lboloni\Documents\Code\_TempData\WBF\ExperimentData\environment\miniberry-100\precalc_ccr\env_value_00025.gz
INFO:root:Loading from gz C:\Users\lboloni\Documents\Code\_TempData\WBF\ExperimentData\environment\miniberry-100\precalc_ccr\env_value_00025.gz done
INFO:root:Environment.proceed - calling the inner_proceed
INFO:root:PrecalculatedEnvironment at timestamp 25
INFO:root:Loading from gz

loading the geometry and environment from saved data
loading done
Saving results to: C:\Users\lboloni\Documents\Code\_TempData\WBF\ExperimentData\nrobot1day\mrmr-lead-2bidders\results.pickle
